# Input matix DEseq2

### Aim
This Notebook generates the count matrix and the column data that need tp be provided as input for the Deseq2 analysis

### Input
Total RNA was extracted from 300ml/100ml Paramecium culture that was subjected to RNAi by feeding. Total RNA was send to Genewiz for mRNA sequencing (poly-A enrichment, NovaSeq 2x150, between 9 Mio and 22 Mio reads per sample). Reads were trimmed for adapters with trim_galore and mapped to the Paramecium tetraurelia strain 51 transcriptome (https://paramecium.i2bc.paris-saclay.fr/files/Paramecium/tetraurelia/51/annotations/ptetraurelia_mac_51/ptetraurelia_mac_51_annotation_v2.0.transcript.fa). Mapping was done with hisat2 allowing 20 multimappings. Using samtools, the properly paired and mapped reads were filtered (-f2 flag) and sorted by the read name (-n flag). Read counts per transcript were acquired with eXpress (https://www.nature.com/articles/nmeth.2251, https://pachterlab.github.io/eXpress/overview.html) with 5 additional online EM rounds to perform on the data after the initial online round (-O 5 flag) to improve accuracy. The eXpress outpur is stored in the results.xprs file

### Note
sample names: AS17 = DevPF2; PS17 = DevPF1

replicates: numbers 1-6

time points: veg = vegetative; 20 = 20-40% fragmentation; 80 = 80-90% fragmentation; 100A = 6h after 80

In [1]:
# import required modules
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from functools import partial, reduce

## Generate the input files for DEseq2

DEseq2 requires two inout files

The **count matrix** contains non-negative integers. The uniq_counts form the resluts.xprs output will be used  
- The index column should be the transcript name (PTET.51.T accession number). 
- The column names should be the samples (e.g. veg_ND7-1)

The **column data** contains information about the samples.
- index column is sample name (e.g. veg_ND7-1). !! It is absolutely cirtical that the columns of the count matrix and the rows of the column data are in the same order !!
- column names are parameters (e.g. KD, replicate, timepoint)

timepoint 50 cannot be included in this analysis because PS17 has no timepoint 50 (50% fragmentation)

### The count matrix

In [17]:
    # retieve sample names from input file
    samples = []
    compna = []
    for line in open(sample_names):
        name = line.split("\n")[0]
        timepoint = name.split("_")[0]
        KD = name.split("_")[1].split("-")[0]
        replicate = name.split("_")[1].split("-")[1]
        compatible_name = f'{KD}_{timepoint}_{replicate}'
        samples.append(name)
        compna.append(compatible_name)
    
    dfs = dict.fromkeys(compna, None)
    

In [31]:
    count = int(-1)
    for i in samples:
        count += 1
        folder = path_data + i + '/eXpress/results.xprs'
        result = pd.read_csv(folder, sep="\t")
        dfs[compna[count]] = result

In [32]:
type(dfs[compna[0]])

pandas.core.frame.DataFrame

In [33]:
def uniq_counts (path_data, sample_names, path_save, name_save, save = False):
    
    '''
    # Prepare the count matrix for deseq2 (rows are samples and colums are the transcripts)
    # contains the number of fragments uniquely mapping to the transcript outputted by eXpress
    '''
    
    # retieve sample names from input file
    samples = []
    compna = []
    for line in open(sample_names):
        name = line.split("\n")[0]
        timepoint = name.split("_")[0]
        KD = name.split("_")[1].split("-")[0]
        replicate = name.split("_")[1].split("-")[1]
        compatible_name = f'{KD}_{timepoint}_{replicate}'
        samples.append(name)
        compna.append(compatible_name)
    
    dfs = dict.fromkeys(compna, None)
    
    # store the results from eXpress as dataframes in dictionary
    count = int(-1)
    for i in samples:
        count += 1
        folder = path_data + i + '/eXpress/results.xprs' # the folder where the eXpress results are saved in
        result = pd.read_csv(folder, sep="\t")
        dfs[compna[count]] = result
         
    # drop all the columns you do not need as input for Deseq2
    for f in compna:     
        dfs[f] = dfs[f].drop(columns=['bundle_id',"tpm",'length','eff_length','tot_counts', 'est_counts',
                                      'ambig_distr_alpha','ambig_distr_beta','fpkm','fpkm_conf_low',
                                      'fpkm_conf_high','solvable','eff_counts'])

    # change the column name uniq_counts to sample name
    for s in compna:
        dfs[s] = dfs[s].rename({'uniq_counts': s}, axis='columns')
        
    # merge the dataframes into one big dataframe and transpose data frame 
    my_reduce = partial(pd.merge, on='target_id', how='outer')                                                              
    uniq_df = reduce(my_reduce, dfs.values())
    uniq_df = uniq_df.set_index(['target_id'])
    
    display(uniq_df)
    
    if save:
        #save as .csv
        save_name = path_save + f"{name_save}_DEseq2_count-matrix.csv"
        uniq_df.to_csv(save_name, sep = ',')
        
    

In [35]:
# set the paths where data is stored and to where output should be saved to
path_data = 
path_save = 

# provide the sample_name file
sample_names = "sample_names.txt"

# provide name to be saved
name_save = "KD489_no50_compatiblenames"

In [36]:
uniq_counts (path_data, sample_names, path_save, name_save, save = True)

,ND7_100A_5,ND7_100A_6,AS17_100A_5,AS17_100A_6,ND7_veg_1,AS17_veg_1,PS17_veg_1,ND7_veg_2,AS17_veg_2,PS17_veg_2,...,PS17_80_3,ND7_80_4,AS17_80_4,PS17_80_4,ND7_100A_3,AS17_100A_3,PS17_100A_3,ND7_100A_4,AS17_100A_4,PS17_100A_4
target_id,,,,,,,,,,,,,,,,,,,,,
PTET.51.1.T0780073,170,127,100,241,173,151,186,150,162,100,...,106,116,101,113,100,130,148,77,105,131
PTET.51.1.T0870117,46,30,45,70,116,88,91,77,108,80,...,53,30,28,55,25,46,57,23,37,59
PTET.51.1.T1400033,5,3,5,11,5,7,4,6,9,8,...,30,3,25,8,3,11,15,4,11,4
PTET.51.1.T0310120,135,135,114,168,110,65,95,100,128,64,...,69,64,75,86,112,88,105,98,77,110
PTET.51.1.T1580010,54,52,67,76,37,32,45,37,44,32,...,19,40,20,32,51,29,34,40,23,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PTET.51.1.T1220104,29,24,26,38,63,44,64,62,67,40,...,13,24,18,21,31,24,23,13,17,13
PTET.51.1.T1570017,79,82,87,98,61,29,47,50,51,27,...,37,45,40,34,65,56,65,59,54,58
PTET.51.1.T0240289,0,1,0,2,14,6,4,6,5,3,...,0,2,1,1,3,1,5,1,0,0


## The column data

In [38]:
def col_data (path_data, sample_names, path_save, name_save, save = False):
    
    '''
    # Prepare the count matrix for deseq2 (rows are samples and colums are the transcripts)
    # contains the number of fragments uniquely mapping to the transcript outputted by eXpress
    '''
    
    # retieve sample names from input file
    header = ['sample','KD','timepoint','replicate']
    columns = []
    
    for line in open(sample_names):
        name = line.split("\n")[0]
        timepoint = name.split("_")[0]
        KD = name.split("_")[1].split("-")[0]
        replicate = name.split("_")[1].split("-")[1]
        compatible_name = f'{KD}_{timepoint}_{replicate}'
        columns.append([compatible_name, KD, timepoint, replicate])
    
    # create data frame out of lists
    df = pd.DataFrame(columns, columns = header)
    df = df.set_index(['sample'])
    
    # add batch column to column data
    # batch effects account for differences in hadling. For example, the samples processed in parallel 
    # on the same day might have another base line than the other samples since the knockdown was differnt 
    # Parallel-processed samples: KD8: AS17-1, AS17-2, PS17-1, and PS17-2 (A); KD9: AS17-3, AS17-4, PS17-3, 
    # and PS17-4 (B); KD4: AS17-5 and AS17-6 (C)

    col         = 'replicate'
    conditions  = [ (df[col] == '1') | (df[col] == '2'), (df[col] == '3') | (df[col] == '4'), 
                   (df[col] == '5') | (df[col] == '6')]
    choices     = [ 'A', 'B', 'C' ]
    
    df["batch"] = np.select(conditions, choices, default=np.nan)
    
    display(df)
    
    if save:
        #save as .csv
        save_name = path_save + f"{name_save}_DEseq2_column-data.csv"
        df.to_csv(save_name, sep = ',')
        
    

In [40]:
col_data (path_data, sample_names, path_save, name_save, save = True)

,KD,timepoint,replicate,batch
sample,,,,
ND7_100A_5,ND7,100A,5,C
ND7_100A_6,ND7,100A,6,C
AS17_100A_5,AS17,100A,5,C
AS17_100A_6,AS17,100A,6,C
ND7_veg_1,ND7,veg,1,A
AS17_veg_1,AS17,veg,1,A
PS17_veg_1,PS17,veg,1,A
ND7_veg_2,ND7,veg,2,A
AS17_veg_2,AS17,veg,2,A
